In [1]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr

# Input Data

In [138]:
P     = 0.5 # GPa
T     = 1000.00 # Deg C
SiO2  = 68.99 # wt%
TiO2  = 0.08
Al2O3 = 19.00
FeO   = 0.05 # For FeO* enter total iron as FeO and Fe2O3 = 0
Fe2O3 = 0
MnO   = 0.01
MgO   = 0.09
CaO   = 0.49
Na2O  = 3.15
K2O   = 7.90
H2O   = 0

Define functions and models

In [139]:
weights = {'SiO2':60.0848, 'TiO2':79.866, 'Al2O3':101.96128, 'FeO':71.8464, 'Fe2O3':159.6922,
           'MnO':70.9374, 'MgO':40.3044, 'CaO':56.0794, 'Na2O':61.97894, 'K2O':94.1954, 
           'H2O':18.0152} # molecular weights
norms   = {'SiO2':1, 'TiO2':1, 'Al2O3':2, 'FeO':1, 'Fe2O3':2, 'MnO':1, 'MgO':1, 'CaO':1, 'Na2O':2, 
           'K2O':2, 'H2O':2} # Normalize to atoms

def WtPerc2MolFracOxide(oxides):
    """Calculate mole fractions from weight percent of oxides.
    
    :Input:
     - *oxides* (ndarray(11)) - Weight percent of oxides.

    :Output:
     - (ndarray(11)) - mole fractions of oxides.
    """
    molFracs = np.zeros_like(oxides)    
    for i, (oxide, weight) in enumerate(weights.items()):
        molFracs[i] = oxides[i]/weight
    molTot = sum(molFracs)
    molFracs /= molTot
    return molFracs, molTot

def WtPerc2MolFrac(oxides):
    """Calculate mole fractions from weight percent of oxides.
    
    :Input:
     - *oxides* (ndarray(11)) - Weight percent of oxides.

    :Output:
     - (ndarray(10)) - mole fractions of elements.
    """
    molFracs = np.zeros_like(oxides)    
    for i, (oxide, weight) in enumerate(weights.items()):
        molFracs[i] = oxides[i]/weight*norms[oxide]
    molFracs /= sum(molFracs)
    molFracs = np.append(np.append(molFracs[0:3],sum(molFracs[3:5])),molFracs[5:])
    return molFracs


def SCAS(P, T, oxides, model='ChowdhuryDasgupta2019'):
    """Calculate Sulfur Content at Anhydrite Saturation (SCAS).
    
    :Input:
     - *P* (float)            - Pressure (GPa).
     - *T* (float)            - Temperature (deg C).
     - *oxides* (ndarray(11)) - Weight percent of oxides.
     - *model* (string)       - Saturation model ('ChowdhuryDasgupta2019' or 'LiRipley2009')

    :Output:
     - (float) - SCAS (ppm).
    """
    
    molFracs, molTot = WtPerc2MolFracOxide(oxides)
    
    if model == 'ChowdhuryDasgupta2019':
        lnXS = -13.23 - 0.50*(10**4/(T + 273.15)) + 3.02*molFracs[0] + 44.28*molFracs[2] + \
        10.14*(molFracs[3]+2*molFracs[4]) + 2.84*molFracs[6] + 36.70*molFracs[7] + 26.27*molFracs[8] - \
        26.27*molFracs[9] + 0.09*oxides[10] - 0.54*np.log(molFracs[7])
        SCAS = np.exp(lnXS)*(molTot + np.exp(lnXS))*32.065*10**4
        
    if model == 'LiRipley2009':
        
        
    return SCAS

In [140]:
SCAS(P, T, [SiO2, TiO2, Al2O3, FeO, Fe2O3, MnO, MgO, CaO, Na2O, K2O, H2O], model='ChowdhuryDasgupta2019')

509.80137343387815